In [6]:
import os
import numpy as np
import pandas as pd
import scipy.io
from scipy.signal import resample
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
# def load_mat(matfile):
#     m = scipy.io.loadmat(matfile)
#     feeg = m['specDat'][0]['fEEG'][0]
#     time = feeg[0]      
#     eeg  = feeg[1].astype(float)
#     sr   = 1.0 / np.diff(time[:2])[0]
#     events = m['szEvents']   # [idx, start_s, end_s, dur_s]
#     return eeg, sr, events

# def to_idx(t, sr):
#     return int(round(t * sr))

# def sliding_windows(arr, win_len):
#     n = len(arr)
#     count = n // win_len
#     return [arr[i*win_len:(i+1)*win_len] for i in range(count)]

In [ ]:
# def process_eeg_data(matfile, window_sec=10.0, resample_hz=None, train_ratio=0.8):
#     eeg, sr, events = load_mat(matfile)
    
#     # optional resampling
#     if resample_hz:
#         new_len = int(len(eeg) * resample_hz / sr)
#         eeg = resample(eeg, new_len)
#         sr = resample_hz

#     W = int(round(window_sec * sr))

#     # build interictal intervals
#     seiz = sorted((to_idx(s,sr), to_idx(e,sr)) for _, s,e,_ in events)
#     inter = []
#     last = 0
#     for s,e in seiz:
#         if s - last >= W:
#             inter.append((last, s))
#         last = e
#     if len(eeg) - last >= W:
#         inter.append((last, len(eeg)))

#     # Windows for inter-ictal periods
#     inter_wins = []
#     for a,b in inter:
#         inter_wins += sliding_windows(eeg[a:b], W)
    
#     # Windows with seizure events at the END
#     seiz_wins = []
#     onset_times = []  # Store onset times relative to window start
    
#     for s,e in seiz:
#         # Check if seizure is long enough to capture a full window
#         seizure_duration = e - s
        
#         if e >= W:
#             # Include window ending at seizure end time (e)
#             window_start = e - W
#             segment = eeg[window_start:e]
            
#             # Calculate onset time relative to window start (in samples)
#             if window_start <= s:  # seizure starts within the window
#                 onset_idx = s - window_start
#                 onset_time = onset_idx / sr  # convert to seconds
#                 onset_times.append(onset_time)
#             else:  # window starts after seizure onset
#                 onset_times.append(0)  # seizure starts before window
                
#             seiz_wins.append(segment)
    
#     # Calculate average onset time within window
#     avg_onset_time = np.mean(onset_times) if onset_times else 0
#     print(f"Average seizure onset time within window: {avg_onset_time:.2f} seconds")
    
#     # balance: down-sample interictal to match seiz count
#     N = min(len(inter_wins), len(seiz_wins))
#     np.random.shuffle(inter_wins)
#     inter_wins = inter_wins[:N]
#     seiz_wins = seiz_wins[:N]
    
#     # Create labels for each window (0 for inter-ictal, 1 for seizure)
#     inter_labels = np.zeros(len(inter_wins))
#     seiz_labels = np.ones(len(seiz_wins))
    
#     # Combine windows and labels
#     all_wins = np.array(inter_wins + seiz_wins)
#     all_labels = np.concatenate([inter_labels, seiz_labels])
    
#     # Create a single indexed array for shuffling
#     indices = np.arange(len(all_wins))
#     np.random.shuffle(indices)
    
#     # Apply the shuffling
#     all_wins = all_wins[indices]
#     all_labels = all_labels[indices]
    
#     # Split into train and test
#     cut = int(train_ratio * len(all_wins))
#     train_wins, test_wins = all_wins[:cut], all_wins[cut:]
#     train_labels, test_labels = all_labels[:cut], all_labels[cut:]
    
#     # Save data
#     os.makedirs('data', exist_ok=True)
#     pd.DataFrame(train_wins).to_csv('data/train.csv', index=False, header=False)
#     pd.DataFrame(test_wins).to_csv('data/test.csv', index=False, header=False)
    
#     # Save labels separately
#     pd.DataFrame(train_labels).to_csv('data/train_labels.csv', index=False, header=False)
#     pd.DataFrame(test_labels).to_csv('data/test_labels.csv', index=False, header=False)
    
#     print(f"train: {len(train_wins)} windows, test: {len(test_wins)} windows")
#     print(f"Seizure windows: {len(seiz_wins)}, Inter-ictal windows: {len(inter_wins)}")
    
#     return {
#         'train_wins': train_wins, 
#         'test_wins': test_wins,
#         'train_labels': train_labels,
#         'test_labels': test_labels,
#         'sr': sr,
#         'window_sec': window_sec,
#         'avg_onset_time': avg_onset_time  # Return the average onset time
#     }

In [ ]:
# def visualize_eeg_data(data_dict):
#     """
#     Create visualizations comparing inter-ictal vs seizure windows
#     with clear indication of the average seizure onset time
#     """
#     train_wins = data_dict['train_wins']
#     test_wins = data_dict['test_wins']
#     train_labels = data_dict['train_labels']
#     test_labels = data_dict['test_labels']
#     sr = data_dict['sr']
#     window_sec = data_dict['window_sec']
#     avg_onset_time = data_dict['avg_onset_time']
    
#     # Create time axis for plotting
#     time_axis = np.linspace(0, window_sec, train_wins.shape[1])
    
#     # Separate windows by type
#     train_inter = train_wins[train_labels == 0]
#     train_seiz = train_wins[train_labels == 1]
#     test_inter = test_wins[test_labels == 0]
#     test_seiz = test_wins[test_labels == 1]
    
#     # Calculate average signals
#     avg_train_inter = np.mean(train_inter, axis=0)
#     avg_train_seiz = np.mean(train_seiz, axis=0)
#     avg_test_inter = np.mean(test_inter, axis=0)
#     avg_test_seiz = np.mean(test_seiz, axis=0)
    
#     # Calculate standard deviation for confidence intervals
#     std_train_inter = np.std(train_inter, axis=0)
#     std_train_seiz = np.std(train_seiz, axis=0)
#     std_test_inter = np.std(test_inter, axis=0)
#     std_test_seiz = np.std(test_seiz, axis=0)
    
#     # Plot Training Data
#     plt.figure(figsize=(15, 10))
    
#     plt.subplot(2, 1, 1)
#     plt.title('Average EEG Signal in Training Set')
#     plt.plot(time_axis, avg_train_inter, 'b-', label='Inter-ictal (n={})'.format(len(train_inter)))
#     plt.fill_between(time_axis, 
#                      avg_train_inter - std_train_inter, 
#                      avg_train_inter + std_train_inter, 
#                      color='b', alpha=0.2)
    
#     plt.plot(time_axis, avg_train_seiz, 'r-', label='Seizure (n={})'.format(len(train_seiz)))
#     plt.fill_between(time_axis, 
#                      avg_train_seiz - std_train_seiz, 
#                      avg_train_seiz + std_train_seiz, 
#                      color='r', alpha=0.2)
    
#     plt.xlabel('Time (seconds)')
#     plt.ylabel('Amplitude')
#     plt.legend()
#     plt.grid(True)
    
#     # Add annotation for seizure onset and end
#     plt.axvline(x=avg_onset_time, color='g', linestyle='--')
#     plt.text(avg_onset_time + 0.1, plt.ylim()[1] * 0.9, 'Avg. Seizure Onset: {:.2f}s'.format(avg_onset_time), 
#              verticalalignment='top')
    
#     plt.axvline(x=window_sec, color='k', linestyle='--')
#     plt.text(window_sec - 1.5, plt.ylim()[1] * 0.8, 'Seizure End: {:.2f}s'.format(window_sec), 
#              verticalalignment='top')
    
#     # Add shaded region from onset to end
#     plt.axvspan(avg_onset_time, window_sec, alpha=0.1, color='r', label='Seizure Region')
    
#     # Plot Testing Data
#     plt.subplot(2, 1, 2)
#     plt.title('Average EEG Signal in Testing Set')
#     plt.plot(time_axis, avg_test_inter, 'b-', label='Inter-ictal (n={})'.format(len(test_inter)))
#     plt.fill_between(time_axis, 
#                      avg_test_inter - std_test_inter, 
#                      avg_test_inter + std_test_inter, 
#                      color='b', alpha=0.2)
    
#     plt.plot(time_axis, avg_test_seiz, 'r-', label='Seizure (n={})'.format(len(test_seiz)))
#     plt.fill_between(time_axis, 
#                      avg_test_seiz - std_test_seiz, 
#                      avg_test_seiz + std_test_seiz, 
#                      color='r', alpha=0.2)
    
#     plt.xlabel('Time (seconds)')
#     plt.ylabel('Amplitude')
#     plt.legend()
#     plt.grid(True)
    
#     # Add annotation for seizure onset and end
#     plt.axvline(x=avg_onset_time, color='g', linestyle='--')
#     plt.text(avg_onset_time + 0.1, plt.ylim()[1] * 0.9, 'Avg. Seizure Onset: {:.2f}s'.format(avg_onset_time), 
#              verticalalignment='top')
    
#     plt.axvline(x=window_sec, color='k', linestyle='--')
#     plt.text(window_sec - 1.5, plt.ylim()[1] * 0.8, 'Seizure End: {:.2f}s'.format(window_sec), 
#              verticalalignment='top')
    
#     # Add shaded region from onset to end
#     plt.axvspan(avg_onset_time, window_sec, alpha=0.1, color='r', label='Seizure Region')
    
#     plt.tight_layout()
#     plt.show()
    
#     # Additional visualization showing individual examples
#     plt.figure(figsize=(15, 10))
    
#     # Plot 10 random examples from each class
#     plt.subplot(2, 2, 1)
#     plt.title('Sample Inter-ictal Windows (Training)')
#     for i in range(min(10, len(train_inter))):
#         plt.plot(time_axis, train_inter[i], alpha=0.5)
#     plt.xlabel('Time (seconds)')
#     plt.ylabel('Amplitude')
    
#     plt.subplot(2, 2, 2)
#     plt.title('Sample Seizure Windows (Training)')
#     for i in range(min(10, len(train_seiz))):
#         plt.plot(time_axis, train_seiz[i], alpha=0.5)
#     plt.xlabel('Time (seconds)')
#     plt.ylabel('Amplitude')
    
#     # Add annotation for seizure onset and end
#     plt.axvline(x=avg_onset_time, color='g', linestyle='--', label='Avg. Seizure Onset')
#     plt.axvline(x=window_sec, color='k', linestyle='--', label='Seizure End')
#     plt.axvspan(avg_onset_time, window_sec, alpha=0.1, color='r', label='Seizure Region')
#     plt.legend()
    
#     plt.subplot(2, 2, 3)
#     plt.title('Sample Inter-ictal Windows (Testing)')
#     for i in range(min(10, len(test_inter))):
#         plt.plot(time_axis, test_inter[i], alpha=0.5)
#     plt.xlabel('Time (seconds)')
#     plt.ylabel('Amplitude')
    
#     plt.subplot(2, 2, 4)
#     plt.title('Sample Seizure Windows (Testing)')
#     for i in range(min(10, len(test_seiz))):
#         plt.plot(time_axis, test_seiz[i], alpha=0.5)
#     plt.xlabel('Time (seconds)')
#     plt.ylabel('Amplitude')
    
#     # Add annotation for seizure onset and end
#     plt.axvline(x=avg_onset_time, color='g', linestyle='--', label='Avg. Seizure Onset')
#     plt.axvline(x=window_sec, color='k', linestyle='--', label='Seizure End')
#     plt.axvspan(avg_onset_time, window_sec, alpha=0.1, color='r', label='Seizure Region')
#     plt.legend()
    
#     plt.tight_layout()
#     plt.show()
    
#     # # Plot power spectrum comparison
#     # plt.figure(figsize=(15, 10))
    
#     # # Calculate power spectrum
#     # def power_spectrum(signals):
#     #     n_samples = signals.shape[1]
#     #     freqs = np.fft.rfftfreq(n_samples, d=1/sr)
#     #     spectra = np.abs(np.fft.rfft(signals, axis=1))**2
#     #     avg_spectrum = np.mean(spectra, axis=0)
#     #     std_spectrum = np.std(spectra, axis=0)
#     #     return freqs, avg_spectrum, std_spectrum
    
#     # # Training set power spectrum
#     # freqs_train_inter, power_train_inter, std_train_inter = power_spectrum(train_inter)
#     # freqs_train_seiz, power_train_seiz, std_train_seiz = power_spectrum(train_seiz)
    
#     # # Testing set power spectrum
#     # freqs_test_inter, power_test_inter, std_test_inter = power_spectrum(test_inter)
#     # freqs_test_seiz, power_test_seiz, std_test_seiz = power_spectrum(test_seiz)
    
#     # # Plot Training Power Spectrum
#     # plt.subplot(2, 1, 1)
#     # plt.title('Power Spectrum - Training Set')
#     # plt.plot(freqs_train_inter, power_train_inter, 'b-', label='Inter-ictal')
#     # plt.fill_between(freqs_train_inter, 
#     #                  power_train_inter - std_train_inter, 
#     #                  power_train_inter + std_train_inter, 
#     #                  color='b', alpha=0.2)
    
#     # plt.plot(freqs_train_seiz, power_train_seiz, 'r-', label='Seizure')
#     # plt.fill_between(freqs_train_seiz, 
#     #                  power_train_seiz - std_train_seiz, 
#     #                  power_train_seiz + std_train_seiz, 
#     #                  color='r', alpha=0.2)
    
#     # plt.xlabel('Frequency (Hz)')
#     # plt.ylabel('Power')
#     # plt.legend()
#     # plt.grid(True)
#     # plt.xlim(0, min(50, sr/2))  # Limit to 0-50 Hz or Nyquist frequency
#     # plt.yscale('log')
    
#     # # Plot Testing Power Spectrum
#     # plt.subplot(2, 1, 2)
#     # plt.title('Power Spectrum - Testing Set')
#     # plt.plot(freqs_test_inter, power_test_inter, 'b-', label='Inter-ictal')
#     # plt.fill_between(freqs_test_inter, 
#     #                  power_test_inter - std_test_inter, 
#     #                  power_test_inter + std_test_inter, 
#     #                  color='b', alpha=0.2)
    
#     # plt.plot(freqs_test_seiz, power_test_seiz, 'r-', label='Seizure')
#     # plt.fill_between(freqs_test_seiz, 
#     #                  power_test_seiz - std_test_seiz, 
#     #                  power_test_seiz + std_test_seiz, 
#     #                  color='r', alpha=0.2)
    
#     # plt.xlabel('Frequency (Hz)')
#     # plt.ylabel('Power')
#     # plt.legend()
#     # plt.grid(True)
#     # plt.xlim(0, min(50, sr/2))  # Limit to 0-50 Hz or Nyquist frequency
#     # plt.yscale('log')
    
#     plt.tight_layout()
#     plt.show()

In [1]:
import os
import numpy as np
import pandas as pd
import scipy.io
from scipy.signal import resample
import matplotlib.pyplot as plt
from tqdm import tqdm

def load_mat(matfile):
    m = scipy.io.loadmat(matfile)
    feeg = m['specDat'][0]['fEEG'][0]
    time = feeg[0]      
    eeg  = feeg[1].astype(float)
    sr   = 1.0 / np.diff(time[:2])[0]
    events = m['szEvents']   # [idx, start_s, end_s, dur_s]
    return eeg, sr, events

def to_idx(t, sr):
    return int(round(t * sr))

def sliding_windows(arr, win_len):
    n = len(arr)
    count = n // win_len
    return [arr[i*win_len:(i+1)*win_len] for i in range(count)]

def visualize_eeg_data(data_dict):
    """Visualize sample EEG data from the processed dataset"""
    # Plot some sample windows
    plt.figure(figsize=(15, 10))
    
    # Plot a few samples from each class
    n_samples = 3
    for i in range(n_samples):
        # Find indices for each class
        seiz_indices = np.where(data_dict['train_labels'] == 1)[0]
        inter_indices = np.where(data_dict['train_labels'] == 0)[0]
        
        if len(seiz_indices) > i and len(inter_indices) > i:
            # Plot seizure sample
            plt.subplot(2, n_samples, i+1)
            plt.plot(data_dict['train_wins'][seiz_indices[i]])
            plt.title(f"Seizure window {i+1}")
            
            # Plot interictal sample
            plt.subplot(2, n_samples, n_samples+i+1)
            plt.plot(data_dict['train_wins'][inter_indices[i]])
            plt.title(f"Interictal window {i+1}")
    
    plt.tight_layout()
    plt.savefig('data/sample_windows.png')
    plt.close()
    
    # Plot class distribution
    plt.figure(figsize=(10, 6))
    train_seizure_count = np.sum(data_dict['train_labels'] == 1)
    train_inter_count = np.sum(data_dict['train_labels'] == 0)
    test_seizure_count = np.sum(data_dict['test_labels'] == 1)
    test_inter_count = np.sum(data_dict['test_labels'] == 0)
    
    plt.bar(['Train Seizure', 'Train Interictal', 'Test Seizure', 'Test Interictal'], 
            [train_seizure_count, train_inter_count, test_seizure_count, test_inter_count])
    plt.title('Class Distribution')
    plt.ylabel('Count')
    plt.savefig('data/class_distribution.png')
    plt.close()

def process_eeg_data(matfile, window_sec=10.0, resample_hz=None, train_ratio=0.8):
    eeg, sr, events = load_mat(matfile)
    
    # optional resampling
    if resample_hz:
        new_len = int(len(eeg) * resample_hz / sr)
        eeg = resample(eeg, new_len)
        sr = resample_hz

    W = int(round(window_sec * sr))

    # build interictal intervals
    seiz = sorted((to_idx(s,sr), to_idx(e,sr)) for _, s,e,_ in events)
    inter = []
    last = 0
    for s,e in seiz:
        if s - last >= W:
            inter.append((last, s))
        last = e
    if len(eeg) - last >= W:
        inter.append((last, len(eeg)))

    # Windows for inter-ictal periods
    inter_wins = []
    for a,b in inter:
        inter_wins += sliding_windows(eeg[a:b], W)
    
    # Windows with seizure events at the END
    seiz_wins = []
    onset_times = []  # Store onset times relative to window start
    
    for s,e in seiz:
        # Check if seizure is long enough to capture a full window
        seizure_duration = e - s
        
        if e >= W:
            # Include window ending at seizure end time (e)
            window_start = e - W
            segment = eeg[window_start:e]
            
            # Calculate onset time relative to window start (in samples)
            if window_start <= s:  # seizure starts within the window
                onset_idx = s - window_start
                onset_time = onset_idx / sr  # convert to seconds
                onset_times.append(onset_time)
            else:  # window starts after seizure onset
                onset_times.append(0)  # seizure starts before window
                
            seiz_wins.append(segment)
    
    # Calculate average onset time within window
    avg_onset_time = np.mean(onset_times) if onset_times else 0
    print(f"Average seizure onset time within window: {avg_onset_time:.2f} seconds")
    
    # Create labels for each window (0 for inter-ictal, 1 for seizure)
    inter_labels = np.zeros(len(inter_wins))
    seiz_labels = np.ones(len(seiz_wins))
    
    return {
        'inter_wins': inter_wins,
        'seiz_wins': seiz_wins,
        'inter_labels': inter_labels,
        'seiz_labels': seiz_labels,
        'sr': sr,
        'window_sec': window_sec,
        'avg_onset_time': avg_onset_time
    }

def process_multiple_files(matfiles, window_sec=5.0, resample_hz=256, train_ratio=0.8):
    """Process multiple mat files and combine the results"""
    
    all_inter_wins = []
    all_seiz_wins = []
    all_inter_labels = []
    all_seiz_labels = []
    
    # Process each file
    for file_path in tqdm(matfiles, desc="Processing files"):
        try:
            result = process_eeg_data(
                matfile=file_path,
                window_sec=window_sec,
                resample_hz=resample_hz,
                train_ratio=train_ratio
            )
            
            # Append results
            all_inter_wins.extend(result['inter_wins'])
            all_seiz_wins.extend(result['seiz_wins'])
            all_inter_labels = np.append(all_inter_labels, result['inter_labels'])
            all_seiz_labels = np.append(all_seiz_labels, result['seiz_labels'])
            
            print(f"Processed {file_path}")
            print(f"  Added {len(result['inter_wins'])} interictal windows")
            print(f"  Added {len(result['seiz_wins'])} seizure windows")
            
        except Exception as e:
            print(f"Error processing {file_path}: {str(e)}")
    
    # Balance the dataset
    n_seiz = len(all_seiz_wins)
    n_inter = len(all_inter_wins)
    
    print(f"Total before balancing - Seizure: {n_seiz}, Interictal: {n_inter}")
    
   
    if n_seiz < n_inter:
        # Down-sample interictal windows
        indices = np.random.choice(n_inter, n_seiz, replace=False)
        all_inter_wins = [all_inter_wins[i] for i in indices]
        all_inter_labels = all_inter_labels[indices]
    elif n_inter < n_seiz:
        # Down-sample seizure windows
        indices = np.random.choice(n_seiz, n_inter, replace=False)
        all_seiz_wins = [all_seiz_wins[i] for i in indices]
        all_seiz_labels = all_seiz_labels[indices]
    
    # Combine windows and labels
    all_wins = np.array(all_inter_wins + all_seiz_wins)
    all_labels = np.concatenate([all_inter_labels, all_seiz_labels])
    
    # Create indices for shuffling
    indices = np.arange(len(all_wins))
    np.random.shuffle(indices)
    
    # Apply shuffling
    all_wins = all_wins[indices]
    all_labels = all_labels[indices]
    
    # Split into train and test sets
    cut = int(train_ratio * len(all_wins))
    train_wins = all_wins[:cut]
    test_wins = all_wins[cut:]
    train_labels = all_labels[:cut]
    test_labels = all_labels[cut:]
    
    # Save data
    os.makedirs('data', exist_ok=True)
    pd.DataFrame(train_wins).to_csv('data/train.csv', index=False, header=False)
    pd.DataFrame(test_wins).to_csv('data/test.csv', index=False, header=False)
    
    # Save labels
    pd.DataFrame(train_labels).to_csv('data/train_labels.csv', index=False, header=False)
    pd.DataFrame(test_labels).to_csv('data/test_labels.csv', index=False, header=False)
    
    # Print statistics
    print("\nDataset Statistics:")
    print(f"Train: {len(train_wins)} windows")
    print(f"Test: {len(test_wins)} windows")
    print(f"Train Seizure: {np.sum(train_labels == 1)} windows")
    print(f"Train Interictal: {np.sum(train_labels == 0)} windows")
    print(f"Test Seizure: {np.sum(test_labels == 1)} windows")
    print(f"Test Interictal: {np.sum(test_labels == 0)} windows")
    
    return {
        'train_wins': train_wins,
        'test_wins': test_wins,
        'train_labels': train_labels,
        'test_labels': test_labels,
        'sr': resample_hz,
        'window_sec': window_sec
    }

In [2]:
# List of mat files
matfiles = [
    r"D:\EEG\Xiaofu\RT\M229_het_PV_RT\EEGCamFP-190716-134143\eeg3_m0-120.mat",
    r"D:\EEG\Xiaofu\RT\M229_het_PV_RT\EEGCamFP-190717-093811\eeg3_m0-119swd.mat",
    r"D:\EEG\Xiaofu\RT\M229_het_PV_RT\EEGCamFP-190717-121140\eeg3_m0-119swd.mat",
    r"D:\EEG\Xiaofu\RT\M233_het_PV_RT\EEGCamFP-190716-154729\eeg3_m0-120swd.mat",
    r"D:\EEG\Xiaofu\RT\M233_het_PV_RT\EEGCamFP-190719-095610\eeg3_m0-240swd.mat",
    r"D:\EEG\Xiaofu\RT\M234_het_PV_RT\EEGCamFP-190717-142155\eeg1_m0-120.mat",
    r"D:\EEG\Xiaofu\RT\M237_het_PV_RT\EEGCamFP-190722-095945\eeg2_m0-180.mat",
    r"D:\EEG\Xiaofu\RT\M237_het_PV_RT\EEGCamFP-190725-093138\eeg2_m0-60.mat",
    r"D:\EEG\Xiaofu\RT\M238_het_PV_RT\EEGCamFP-190722-133936\eeg1_m0-180bin0.5step0.2.mat",
    r"D:\EEG\Xiaofu\RT\M238_het_PV_RT\EEGCamFP-190731-234746\eeg1_m0-180.mat",
    r"D:\EEG\Xiaofu\VPM\M710_het_CaMKII\EEGCamFP20210322-210323-185103\eeg3_m0-60.mat",
    r"D:\EEG\Xiaofu\VPM\M710_het_CaMKII\EEGCamFP20210322-210323-205108\eeg3_m0-59.mat",
    r"D:\EEG\Xiaofu\VPM\M710_het_CaMKII\EEGCamFP20210322-210324-005115\eeg3_m0-59.mat",
    r"D:\EEG\Xiaofu\VPM\M710_het_CaMKII\EEGCamFP20210322-210324-025115\eeg3_m0-59.mat",
    r"D:\EEG\Xiaofu\VPM\M710_het_CaMKII\EEGCamFP20210322-210324-045108\eeg3_m0-59.mat",
    r"D:\EEG\Xiaofu\VPM\M711_het_CaMKII\EEGCamFP20210322-210323-100654\eeg3_m0-60_phot3dots.mat",
    r"D:\EEG\Xiaofu\VPM\M711_het_CaMKII\EEGCamFP20210322-210323-113757\eeg3_m0-60.mat",
    r"D:\EEG\Xiaofu\VPM\M711_het_CaMKII\EEGCamFP20210322-210323-130857\eeg3_m0-60.mat",
    r"D:\EEG\Xiaofu\VPM\M711_het_CaMKII\EEGCamFP20210322-210323-161057\eeg3_m0-60.mat",
    r"D:\EEG\Xiaofu\VPM\M1079CaMKII\EEG_FP_Sasa-220314-084728\eeg3_m0-60.mat",
    r"D:\EEG\Xiaofu\VPM\M1079CaMKII\EEG_FP_Sasa-220314-101740\eeg3_m0-59.mat",
    r"D:\EEG\Xiaofu\VPM\M1079CaMKII\EEG_FP_Sasa-220314-114735\eeg3_m0-59.mat",
    r"D:\EEG\Xiaofu\VPM\M1079CaMKII\EEG_FP_Sasa-220314-190244\eeg3_m0-59.5.mat",
    r"D:\EEG\Xiaofu\VPM\M1080CaMKII\EEG_FP_Sasa-220315-122317\eeg3_m0-60.mat",
    r"D:\EEG\Xiaofu\VPM\M1080CaMKII\EEG_FP_Sasa-220315-142326\eeg3_m0-59.5.mat",
    r"D:\EEG\Xiaofu\VPM\M1080CaMKII\EEG_FP_Sasa-220315-162328\eeg3_m0-59.5.mat",
    r"D:\EEG\Xiaofu\VPM\M1080CaMKII\EEG_FP_Sasa-220316-062328\eeg3_m0-59.5.mat",
    r"D:\EEG\Xiaofu\VPM\M1080CaMKII\EEG_FP_Sasa-220316-082323\eeg3_m0-59.5.mat"
]

# Process all files
data_dict = process_multiple_files(
    matfiles=matfiles,
    window_sec=5.0,
    resample_hz=256,
    train_ratio=0.8
)

# Visualize the results
visualize_eeg_data(data_dict)

print("Processing complete! Data saved to the 'data' directory.")







# # Process the data
# data_dict = process_eeg_data(
#     matfile=matfile,
#     window_sec=5.0,
#     resample_hz=256,  # Set to a value if you want to resample
#     train_ratio=0.8
# )

# # Visualize the results
# visualize_eeg_data(data_dict)

Processing files:   4%|▎         | 1/28 [00:02<01:10,  2.60s/it]

Average seizure onset time within window: 3.07 seconds
Processed D:\EEG\Xiaofu\RT\M229_het_PV_RT\EEGCamFP-190716-134143\eeg3_m0-120.mat
  Added 1381 interictal windows
  Added 62 seizure windows


Processing files:   7%|▋         | 2/28 [00:04<00:58,  2.24s/it]

Average seizure onset time within window: 2.73 seconds
Processed D:\EEG\Xiaofu\RT\M229_het_PV_RT\EEGCamFP-190717-093811\eeg3_m0-119swd.mat
  Added 1347 interictal windows
  Added 77 seizure windows


Processing files:  11%|█         | 3/28 [00:06<00:50,  2.03s/it]

...

Processing files:  96%|█████████▋| 27/28 [00:38<00:00,  1.06it/s]

Average seizure onset time within window: 3.45 seconds
Processed D:\EEG\Xiaofu\VPM\M1080CaMKII\EEG_FP_Sasa-220316-062328\eeg3_m0-59.5.mat
  Added 690 interictal windows
  Added 30 seizure windows


Processing files: 100%|██████████| 28/28 [00:39<00:00,  1.40s/it]

Average seizure onset time within window: 3.58 seconds
Processed D:\EEG\Xiaofu\VPM\M1080CaMKII\EEG_FP_Sasa-220316-082323\eeg3_m0-59.5.mat
  Added 652 interictal windows
  Added 75 seizure windows
Total before balancing - Seizure: 2785, Interictal: 27811



Dataset Statistics:
Train: 4456 windows
Test: 1114 windows
Train Seizure: 2214 windows
Train Interictal: 2242 windows
Test Seizure: 571 windows
Test Interictal: 543 windows
Processing complete! Data saved to the 'data' directory.


In [3]:
# Visualize the results
visualize_eeg_data(data_dict)

print("Processing complete! Data saved to the 'data' directory.")

Processing complete! Data saved to the 'data' directory.
